In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path

# Classifiers
from sklearn.naive_bayes import GaussianNB # Gaussiana de Naive-Bayes
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron
from sklearn.neighbors import KNeighborsClassifier # K-Nearest Neighbors
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.ensemble import AdaBoostClassifier # AdaBoost
from sklearn.svm import SVC # Support Vector Machine (Linear, Polynomial, RBF)

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
from sklearn.metrics import f1_score

In [28]:
datasets_name_list = ['Raw', 'Normalized', 'Fourier', 'HOS', 'SCM']

classifiers_name_list = ['Naive_Bayes','MLP','Nearest_Neighbors','Random_Forest','SVM_Linear','SVM_Polynomial','SVM_RBF']

metrics_name_list = ['accuracy', 'precision', 'sensitivity', 'specificity', 'f1_score']

In [29]:
# Load datasets
RawDs = pd.read_csv(path.join("..","results","extraction","32FINAL.csv"))
NormDs = pd.read_csv(path.join("..","results","extraction","32FINAL_NORM.csv"))
FourierDs = pd.read_csv(path.join("..","results","extraction","32FINAL_Fourier.csv"))
HOSDs = pd.read_csv(path.join("..","results","extraction","32FINAL_HOS.csv"))
SCMDs = pd.read_csv(path.join("..","results","extraction","32FINAL_SCM.csv"))

# Exclude Tmp columns
RawDs = RawDs.drop(columns=['Tmp'+str(i) for i in range(100)])
NormDs = NormDs.drop(columns=['Tmp'+str(i) for i in range(100)])

# Put it into a list
datasets_list = [RawDs, NormDs, FourierDs, HOSDs, SCMDs]

In [53]:
from scipy.stats import randint

# Load classifiers
classifiers_list = [GaussianNB(),        
                    MLPClassifier(max_iter=1000, solver='adam', learning_rate_init=5e-04),      
                    KNeighborsClassifier(),   
                    RandomForestClassifier(),
                    SVC(kernel='linear', probability=True, max_iter=3000, tol=1e-3),
                    SVC(kernel='poly', probability=True, max_iter=3000, tol=1e-3),
                    SVC(kernel='rbf', probability=True, max_iter=3000, tol=1e-3)
                   ]

param_dist_dict = {'Naive_Bayes': [], 
                   'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))},
                   'Nearest_Neighbors': {"n_neighbors": [1,3,5,7,9,11]}, 
                   'Random_Forest': {"n_estimators": [3000],
                                     "max_depth": [6, None],
                                     "max_features": randint(1, 11),
                                     "min_samples_split": randint(2, 11),
                                     "min_samples_leaf": randint(1, 11),
                                     "bootstrap": [True, False],
                                     "criterion": ["gini", "entropy"]},
                    'SVM_Linear': {'kernel': ['linear'], 'C': [2**i for i in range(-5,15)]},
                    'SVM_Polynomial': {'kernel': ['poly'], 'degree': [3, 5, 7 ,9], 'C': [2**i for i in range(-5,15)]},                    
                    'SVM_RBF': {'kernel': ['rbf'], 'gamma': [2**i for i in range(-15,3)],
                                'C': [2**i for i in range(-5,15)]}
                  }

In [54]:
from sklearn.model_selection import RandomizedSearchCV

for n_ds, dataset in enumerate(datasets_list):
    X, y = dataset.iloc[:,:-1], dataset.iloc[:,-1]
    random_search = dict((k,[]) for k in classifiers_name_list)
    print(random_search)
    for n_clf, classifier in enumerate(classifiers_list):
        classifier_name = classifiers_name_list[n_clf]
        print(classifier_name)
        if n_clf != 0:
            random_search[classifier_name] = RandomizedSearchCV(classifier, param_dist_dict[classifier_name], cv=3, 
                                                                n_iter=5, verbose=5, scoring='accuracy')
            random_search[classifier_name].fit(X, y)
    print("random search: {}".format(random_search))  

{'SVM_Polynomial': [], 'MLP': [], 'SVM_Linear': [], 'Naive_Bayes': [], 'SVM_RBF': [], 'Nearest_Neighbors': [], 'Random_Forest': []}
Naive_Bayes
MLP
Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] hidden_layer_sizes=344 ..........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .............. hidden_layer_sizes=344, score=0.651, total=   1.6s
[CV] hidden_layer_sizes=344 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] .............. hidden_layer_sizes=344, score=0.597, total=   1.5s
[CV] hidden_layer_sizes=344 ..........................................


...

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] .... gamma=0.125, kernel=rbf, C=16384, score=1.000, total=   0.1s
[CV] gamma=0.125, kernel=rbf, C=16384 ................................
[CV] .... gamma=0.125, kernel=rbf, C=16384, score=0.995, total=   0.1s
[CV] gamma=2, kernel=rbf, C=256 ......................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


[CV] .......... gamma=2, kernel=rbf, C=256, score=1.000, total=   0.2s
[CV] gamma=2, kernel=rbf, C=256 ......................................
[CV] .......... gamma=2, kernel=rbf, C=256, score=1.000, total=   0.2s
[CV] gamma=2, kernel=rbf, C=256 ......................................
[CV] .......... gamma=2, kernel=rbf, C=256, score=0.989, total=   0.2s
[CV] gamma=0.03125, kernel=rbf, C=16 .................................
[CV] ..... gamma=0.03125, kernel=rbf, C=16, score=0.995, total=   0.0s
[CV] gamma=0.03125, kernel=rbf, C=16 .................................
[CV] ..... gamma=0.03125, kernel=rbf, C=16, score=0.995, total=   0.0s
[CV] gamma=0.03125, kernel=rbf, C=16 .................................
[CV] ..... gamma=0.03125, kernel=rbf, C=16, score=0.986, total=   0.0s
[CV] gamma=0.0625, kernel=rbf, C=16 ..................................
[CV] ...... gamma=0.0625, kernel=rbf, C=16, score=0.995, total=   0.0s
[CV] gamma=0.0625, kernel=rbf, C=16 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.2s finished
